<a href="https://colab.research.google.com/github/NathVM/GA/blob/main/Neo4JGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Measuring performance of Graph Analytics Algorithms using Node4j graphs

---



Setup:

---



In [ ]:
!pip install neo4j

Imports:

---



In [2]:
import pandas as pd
from neo4j import GraphDatabase, basic_auth

Neo4j connection:

---



In [3]:
driver = GraphDatabase.driver(
  "bolt://44.203.104.228:7687",
  auth=basic_auth("neo4j", "rollouts-orifices-basin"))



---



In [7]:
def run_query(query, parameters = {}):
    with driver.session(database="neo4j") as session:
        result = session.run(query,parameters)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())


cypher_query = '''
MATCH (n)
RETURN COUNT(n) AS count
LIMIT $limit
'''

<ipython-input-7-0d6fdcc241b1>:13: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


0


Dataset: 